# 导入模块

In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import seaborn as sns 
#sns.set(style="darkgrid")
plt.rcParams["font.sans-serif"]=["SimHei"]
plt.rcParams["axes.unicode_minus"]=False


from dateutil.relativedelta import relativedelta
from datetime import datetime as dt
import time

import warnings  
warnings.filterwarnings("ignore")

import sys
import pymysql
from sqlalchemy import create_engine
import json
import datetime 
from datetime import timedelta ,time 
import re

# 连接数据库

In [8]:
def query(sql,
          host="rr-wz9wx0w3yti9d4f6wro.mysql.rds.aliyuncs.com",
          user="ylc",
          password="1O8t5lcJ5aMhwwPEUUjS",
          database = '' ,
          port=3306
         ):   
    conn = pymysql.connect(
        host=host,
        user=user,
        port=port,
        password=password,
        max_allowed_packet=1073741824,
        charset="utf8")
    try:
        df = pd.read_sql(sql, con=conn)
        conn.close()
    except:
        print('error')
        conn.close()
        raise
    return df

# 导入数据

In [9]:
sql = '''
SELECT DATE_FORMAT(om.create_time, '%Y-%m-%d') 日期,
count(distinct om.order_number) 进件
,sum(case when om.`status` in (4,5,6,8,15) THEN 1 else 0 end) 出库
,concat(round(sum(case when om.`status` in (4,5,6,8,15) THEN 1 else 0 end)/count(distinct om.order_number)*100, 2), '%') 进件出库率
FROM db_digua_business.t_order om 
left join db_digua_business.t_platform_activity tpa on tpa.id=om.activity_id
WHERE om.create_time>='2024-09-01' and tpa.type=4
GROUP BY DATE_FORMAT(om.create_time, '%Y-%m-%d')
'''
df = query(sql)

In [10]:
sql2 = '''
select 
om.create_time 下单时间, om.order_number 订单号, om.status 状态, om.user_mobile 下单手机号, tobi.order_bank_phone 号卡手机号, tprm.model_number 机型, tod.product_name 机型1, tmu.true_name 姓名
from db_digua_business.t_order om
left join db_digua_business.t_postlease_receivables_monitoring tprm on tprm.order_id=om.id
left join db_digua_business.t_member_user tmu on tmu.id=om.user_id
left join db_digua_business.t_order_details tod on tod.order_id=om.id
left join db_digua_business.t_platform_activity tpa on tpa.id=om.activity_id
left join db_digua_business.t_order_bank_info tobi on tobi.order_id=om.id
where om.create_time>='2024-09-01' and tpa.type=4 and om.status=2
'''
df2 = query(sql2)

In [11]:
df[-8:].to_excel('F:/需求/力哥需求/联通号卡数据.xlsx', index=False)

In [12]:
# df2.to_excel('F:/宏生/联通号卡在租.xlsx', index=False)
# df2.to_excel('F:/宏生/联通号卡待发货.xlsx', index=False)